In [23]:
from math import *
import random
import pickle
import numpy as np

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


In [24]:
class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s


In [34]:
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s

def UCT1(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            #state.DoMove(random.choice(state.GetMoves()))            
            model_dt = pickle.load(open("dt11.pickle","rb"))
            np_array=np.asarray(state.board)
            move=model_dt.predict(np_array.reshape(1,-1))
            #print(move)
            if move[0] in state.GetMoves():
                state.DoMove(move[0])
            else:
                state.DoMove(random.choice(state.GetMoves()))
            

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    #if verbose: print(rootnode.TreeToString(0))
    #else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

#___________________-------__________________________________________________________________________________________

def UCT2(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            #state.DoMove(random.choice(state.GetMoves()))            
            model_dt = pickle.load(open("dt10.pickle","rb"))
            np_array=np.asarray(state.board)
            move=model_dt.predict(np_array.reshape(1,-1))
            #print(move)
            if move[0] in state.GetMoves():
                state.DoMove(move[0])
            else:
                state.DoMove(random.choice(state.GetMoves()))
            

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    #if verbose: print(rootnode.TreeToString(0))
    #else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited


def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    num1 = 0 
    state = OXOState() # uncomment to play OXO
    while state.GetMoves() != []:
        #print(str(state))
        board=state.board
        player=state.playerJustMoved
        
        if state.playerJustMoved == 1:
            m = UCT1(rootstate=state, itermax=250, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT2(rootstate=state, itermax=250, verbose=False)
        #print("Best Move: " + str(m) + "\n")
        
                       
        board1 = str(board).strip('[]')
        
       # with open('data_10.csv','a+') as f:
       #     f.write(str(player)+',')
       #     f.write(board1)
       #     f.write(','+str(m))
       #     f.write('\n')
       # f.close
        
        
        #with open('data_10.csv','a+') as f:
        #    f.write(str(player)+',')
        #    f.write(board1)
        #    f.write(','+str(m))
        #    f.write('\n')
        #f.close        
                
        state.DoMove(m)
        if state.GetResult(state.playerJustMoved) != False:
            #print(str(state))
            break
            
                           
    if state.GetResult(state.playerJustMoved) == 1.0:
        #print("Player " + str(state.playerJustMoved) + " wins!")
        # The Final classifier( classifier 11 on the dataset data_csv.10)- Agent is winning
        num1 = state.playerJustMoved
    elif state.GetResult(state.playerJustMoved) == 0.0:
        #print("Player " + str(3 - state.playerJustMoved) + " wins!")
        # The other classifiers (on datasets oxo-game to data_9)- agent is winning
        num1 = 3 - state.playerJustMoved
    return num1   

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    mynum=0
    pl1=0
    pl2=0
    draw=0
    lst1=[]
    for i in range(100):
        #print('i is '+str(i))
        mynum=UCTPlayGame()
        if mynum==1:
            # The Hybrid Agent has won           
            pl1 = pl1+1
        elif mynum==2:
            # The MCTS agaent has won
            pl2 = pl2+1
        else:
            # Its a draw
            draw = draw+1
            
    print(pl1,pl2,draw)
    
    with open('FinalCompetition1.csv','a+') as f1:
        f1.write(str(pl1)+',')
        f1.write(str(pl2)+',')
        f1.write(str(draw))
        f1.write('\n')
    f1.close  


12 1 87




# Code to generate the dataset and storing it in oxo-game.csv


In [33]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np


In [34]:
oxo_df=pd.read_csv("oxo-game.csv",header=None)

In [35]:
oxo_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,2,0,0,0,0,0,0,0,0,0,4
1,1,0,0,0,0,1,0,0,0,0,0
2,2,2,0,0,0,1,0,0,0,0,2
3,1,2,0,1,0,1,0,0,0,0,6
4,2,2,0,1,0,1,0,2,0,0,3


In [252]:
oxo_df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64')

In [36]:
x=oxo_df.iloc[:,1:10]
y=oxo_df.iloc[:,10]

In [254]:
x.head(),y.head()

(   1  2  3  4  5  6  7  8  9
 0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  1  0  0  0  0
 2  2  0  0  0  1  0  0  0  0
 3  2  0  1  0  1  0  0  0  0
 4  2  0  1  0  1  0  2  0  0, 0    4
 1    0
 2    2
 3    6
 4    3
 Name: 10, dtype: int64)

In [37]:
X_train,X_test,Y_train,Y_test=train_test_split(x, y, test_size=0.20,random_state=20,stratify=y)

# Training a Decision tree classifier on the oxonew-game.csv dataset generated by the mcts code and storing the classifier in a pickle file dt1.pickle.

In [38]:
parameters_dt = {'criterion':('gini', 'entropy'),
'max_depth':range(2, 10),
'class_weight':('balanced', None),
'presort':(False,True),
}

dt = DecisionTreeClassifier()
clf_dt = GridSearchCV(dt, parameters_dt,cv = 5, scoring = 'accuracy')
clf_dt1=clf_dt.fit(X_train, Y_train)
def display(results):
    print(f'the best results are :{results.best_params_}')
display(clf_dt)
dt_pred=clf_dt1.predict(X_test)
accuracy_score(Y_test,dt_pred)

the best results are :{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 9, 'presort': False}


0.7386363636363636

In [39]:
tree = DecisionTreeClassifier(max_depth=9,criterion='entropy',presort= False,class_weight='balanced')
clf_dt=tree.fit(X_train, Y_train)
dt_pred = clf_dt.predict(X_test)
dt_pred
accuracy_dt=round(accuracy_score(Y_test, dt_pred)*100,2)
print("Accuracy with Pruned Decision Tree Classifier :"+ str(accuracy_dt)+" %")
kappa_stats=round(metrics.cohen_kappa_score(dt_pred,Y_test),2)
print(kappa_stats)

Accuracy with Pruned Decision Tree Classifier :73.86 %
0.71


In [40]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 

In [42]:
results=confusion_matrix(Y_test, dt_pred)

In [43]:
print('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(Y_test, dt_pred))
print('Report : ')
print(classification_report(Y_test, dt_pred))

Confusion Matrix :
[[71  0  0  3 25  0  1  0  0]
 [ 0 59  4  0 12  2 14  7  0]
 [ 5  1 59  1  9  0 21  1  0]
 [ 0  1  3 56  6 15 12  3  1]
 [12  0  0  0 85  1  0  2  0]
 [ 1  7  0  1  2 79  1  1  4]
 [ 1  0  0  3 14  0 80  0  0]
 [ 0  5  1  0  2  0  3 85  0]
 [ 1  1  0  1  4  5  1  9 76]]
Accuracy Score : 0.7386363636363636
Report : 
              precision    recall  f1-score   support

           0       0.78      0.71      0.74       100
           1       0.80      0.60      0.69        98
           2       0.88      0.61      0.72        97
           3       0.86      0.58      0.69        97
           4       0.53      0.85      0.66       100
           5       0.77      0.82      0.80        96
           6       0.60      0.82      0.69        98
           7       0.79      0.89      0.83        96
           8       0.94      0.78      0.85        98

    accuracy                           0.74       880
   macro avg       0.77      0.74      0.74       880
weighted avg  

In [264]:
import pickle
pickle_out = open("dt1.pickle","wb")
pickle.dump(clf_dt, pickle_out)
pickle_out.close()

In [265]:
#pickle_in = open("dt1.pickle","rb")
#example_dt1 = pickle.load(pickle_in)

In [266]:
#print(example_dt1)
#pickle_out.close()

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=9,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=True,
                       random_state=20, splitter='best')


In [267]:
with open('dt-classifier.csv','a+') as f1:
    f1.write('dt1.pickle' +',')
    f1.write(str(accuracy_dt)+',')
    f1.write(str(kappa_stats))
    f1.write('\n')
f1.close 

<function TextIOWrapper.close()>

# After creating the first pickle file we need to modify the rollout function and use the decision tree classfier on dt1.pickle to find the best move and doing a random serach when unable to find. Generating the dataset data_1.csv for 100 games. we will do this for 10 games and create 10 pickle file containing the decision tree classifier and 10 dataset files.

In [44]:
dt2=pd.read_csv("data_1.csv",header=None)
x=dt2.iloc[:,1:10]
y=dt2.iloc[:,10]

In [279]:
x.head(),y.head()

(   1  2  3  4  5  6  7  8  9
 0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0
 2  1  0  0  0  2  0  0  0  0
 3  1  0  0  0  2  1  0  0  0
 4  1  0  0  0  2  1  2  0  0, 0    0
 1    4
 2    5
 3    6
 4    2
 Name: 10, dtype: int64)

In [45]:
X_train,X_test,Y_train,Y_test=train_test_split(x, y, test_size=0.20,random_state=20,stratify=y)

In [46]:
parameters_dt = {'criterion':('gini', 'entropy'),
'max_depth':range(2,10),
'class_weight':('balanced', None),
'presort':(False,True),
}

dt = DecisionTreeClassifier()
clf_dt = GridSearchCV(dt, parameters_dt,cv = 5, scoring = 'accuracy')
clf_dt1=clf_dt.fit(X_train, Y_train)
def display(results):
    print(f'the best results are :{results.best_params_}')
display(clf_dt)
dt_pred=clf_dt1.predict(X_test)
accuracy_score(Y_test,dt_pred)

the best results are :{'class_weight': None, 'criterion': 'entropy', 'max_depth': 9, 'presort': False}


C:\Users\Dilba\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.8547486033519553

In [47]:
tree = DecisionTreeClassifier(max_depth=9,criterion='entropy',presort=False,class_weight=None)
clf_dt=tree.fit(X_train, Y_train)
dt_pred = clf_dt.predict(X_test)
dt_pred
accuracy_dt=round(accuracy_score(Y_test, dt_pred)*100,2)
print("Accuracy with Pruned Decision Tree Classifier :"+ str(accuracy_dt)+" %")
kappa_stats=round(metrics.cohen_kappa_score(dt_pred,Y_test),2)
print(kappa_stats)

Accuracy with Pruned Decision Tree Classifier :85.47 %
0.84


In [283]:

pickle_out = open("dt2.pickle","wb")
pickle.dump(clf_dt, pickle_out)
pickle_out.close()

In [284]:
with open('dt-classifier.csv','a+') as f1:
    f1.write('dt2.pickle' +',')
    f1.write(str(accuracy_dt)+',')
    f1.write(str(kappa_stats))
    f1.write('\n')
f1.close 

<function TextIOWrapper.close()>

# Training the Decision tree classifier on the data2.csv dataset generated by the decision tree classifier2 and storing the classifier in a pickle file dt3.pickle.

In [48]:
dt3=pd.read_csv("data_2.csv",header=None)
x=dt3.iloc[:,1:10]
y=dt3.iloc[:,10]

In [367]:
X_train,X_test,Y_train,Y_test=train_test_split(x, y, test_size=0.20,random_state=20,stratify=y)

In [49]:
parameters_dt = {'criterion':('gini', 'entropy'),
'max_depth':range(2,10),
'class_weight':('balanced', None),
'presort':(False,True),
}

dt = DecisionTreeClassifier()
clf_dt = GridSearchCV(dt, parameters_dt,cv = 5, scoring = 'accuracy')
clf_dt1=clf_dt.fit(X_train, Y_train)
def display(results):
    print(f'the best results are :{results.best_params_}')
display(clf_dt)
dt_pred=clf_dt1.predict(X_test)
accuracy_score(Y_test,dt_pred)

the best results are :{'class_weight': None, 'criterion': 'gini', 'max_depth': 9, 'presort': False}


C:\Users\Dilba\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.8547486033519553

In [50]:
tree = DecisionTreeClassifier(max_depth=9,criterion='gini',presort= False,class_weight=None)
clf_dt=tree.fit(X_train, Y_train)
dt_pred = clf_dt.predict(X_test)
dt_pred
accuracy_dt=round(accuracy_score(Y_test, dt_pred)*100,2)
print("Accuracy with Pruned Decision Tree Classifier :"+ str(accuracy_dt)+" %")
kappa_stats=round(metrics.cohen_kappa_score(dt_pred,Y_test),2)
print(kappa_stats)

Accuracy with Pruned Decision Tree Classifier :85.47 %
0.84


In [290]:

pickle_out = open("dt3.pickle","wb")
pickle.dump(clf_dt, pickle_out)
pickle_out.close()

In [291]:
with open('dt-classifier.csv','a+') as f1:
    f1.write('dt3.pickle' +',')
    f1.write(str(accuracy_dt)+',')
    f1.write(str(kappa_stats))
    f1.write('\n')
f1.close

<function TextIOWrapper.close()>

In [51]:
results=confusion_matrix(Y_test, dt_pred)
print('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(Y_test, dt_pred))
print('Report : ')
print(classification_report(Y_test, dt_pred))



Confusion Matrix :
[[12  0  0  0  8  0  0  0  0]
 [ 0 18  0  2  0  0  0  0  0]
 [ 0  0 18  0  0  0  1  0  0]
 [ 0  0  4 16  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0]
 [ 0  0  2  3  0 15  0  0  0]
 [ 1  0  1  0  1  0 17  0  0]
 [ 0  0  0  0  0  0  0 19  1]
 [ 0  0  0  0  0  2  0  0 18]]
Accuracy Score : 0.8547486033519553
Report : 
              precision    recall  f1-score   support

           0       0.92      0.60      0.73        20
           1       1.00      0.90      0.95        20
           2       0.72      0.95      0.82        19
           3       0.76      0.80      0.78        20
           4       0.69      1.00      0.82        20
           5       0.88      0.75      0.81        20
           6       0.94      0.85      0.89        20
           7       1.00      0.95      0.97        20
           8       0.95      0.90      0.92        20

    accuracy                           0.85       179
   macro avg       0.87      0.86      0.85       179
weighted avg  

In [52]:
dt9=pd.read_csv("data_8.csv",header=None)
x=dt9.iloc[:,1:10]
y=dt9.iloc[:,10]

In [53]:
x.head()

,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0
2,0,0,1,0,2,0,0,0,0
3,1,0,1,0,2,0,0,0,0
4,1,2,1,0,2,0,0,0,0


In [54]:

y.head()

0    2
1    4
2    0
3    1
4    7
Name: 10, dtype: int64

In [55]:
X_train,X_test,Y_train,Y_test=train_test_split(x, y, test_size=0.20,random_state=20,stratify=y)

In [56]:
parameters_dt = {'criterion':('gini', 'entropy'),
                 'max_depth':range(2,10),
'class_weight':('balanced', None),
'presort':(False,True),
}

dt = DecisionTreeClassifier()
clf_dt = GridSearchCV(dt, parameters_dt,cv = 5, scoring = 'accuracy')
clf_dt1=clf_dt.fit(X_train, Y_train)
def display(results):
    print(f'the best results are :{results.best_params_}')
display(clf_dt)
dt_pred=clf_dt1.predict(X_test)
accuracy_score(Y_test,dt_pred)

the best results are :{'class_weight': None, 'criterion': 'entropy', 'max_depth': 9, 'presort': False}


C:\Users\Dilba\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.7816091954022989

In [57]:
tree = DecisionTreeClassifier(max_depth=9,criterion='entropy',presort= False,class_weight=None)
clf_dt=tree.fit(X_train, Y_train)
dt_pred = clf_dt.predict(X_test)
dt_pred
accuracy_dt=round(accuracy_score(Y_test, dt_pred)*100,2)
print("Accuracy with Pruned Decision Tree Classifier :"+ str(accuracy_dt)+" %")
kappa_stats=round(metrics.cohen_kappa_score(dt_pred,Y_test),2)
print(kappa_stats)

Accuracy with Pruned Decision Tree Classifier :78.16 %
0.75


In [350]:
pickle_out = open("dt9.pickle","wb")
pickle.dump(clf_dt, pickle_out)
pickle_out.close()

In [351]:
with open('dt-classifier.csv','a+') as f1:
    f1.write('dt9.pickle' +',')
    f1.write(str(accuracy_dt)+',')
    f1.write(str(kappa_stats))
    f1.write('\n')
f1.close

<function TextIOWrapper.close()>

In [58]:
results=confusion_matrix(Y_test, dt_pred)
print('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(Y_test, dt_pred))
print('Report : ')
print(classification_report(Y_test, dt_pred))


Confusion Matrix :
[[19  0  0  0  0  0  1  0  0]
 [ 0 18  0  0  0  0  0  1  0]
 [ 0  1 10  0  0  0  8  0  0]
 [ 1  3  1 13  0  0  1  0  0]
 [ 0  0  0  0 13  0  7  0  0]
 [ 0  0  1  2  0 15  0  0  1]
 [ 0  1  0  1  0  0 17  0  0]
 [ 0  4  0  1  0  0  0 13  1]
 [ 0  1  1  0  0  0  0  0 18]]
Accuracy Score : 0.7816091954022989
Report : 
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        20
           1       0.64      0.95      0.77        19
           2       0.77      0.53      0.62        19
           3       0.76      0.68      0.72        19
           4       1.00      0.65      0.79        20
           5       1.00      0.79      0.88        19
           6       0.50      0.89      0.64        19
           7       0.93      0.68      0.79        19
           8       0.90      0.90      0.90        20

    accuracy                           0.78       174
   macro avg       0.83      0.78      0.78       174
weighted avg  

In [59]:
dt11=pd.read_csv("data_10.csv",header=None)
x=dt11.iloc[:,1:10]
y=dt11.iloc[:,10]

In [60]:
x.head()

,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,2,0,0,0,0
3,1,1,0,0,2,0,0,0,0
4,1,1,2,0,2,0,0,0,0


In [61]:
y.head()

0    0
1    4
2    1
3    2
4    6
Name: 10, dtype: int64

In [62]:
X_train,X_test,Y_train,Y_test=train_test_split(x, y, test_size=0.20,random_state=20,stratify=y)

In [63]:
parameters_dt = {'criterion':('gini', 'entropy'),
'max_depth':range(2,10),
'class_weight':('balanced', None),
'presort':(False,True),
}

dt = DecisionTreeClassifier()
clf_dt = GridSearchCV(dt, parameters_dt,cv = 5, scoring = 'accuracy')
clf_dt1=clf_dt.fit(X_train, Y_train)
def display(results):
    print(f'the best results are :{results.best_params_}')
display(clf_dt)
dt_pred=clf_dt1.predict(X_test)
accuracy_score(Y_test,dt_pred)

the best results are :{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 9, 'presort': True}


C:\Users\Dilba\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.7687861271676301

In [64]:
tree = DecisionTreeClassifier(max_depth=9,criterion='entropy',presort= True,class_weight='balanced', random_state=20)
clf_dt=tree.fit(X_train, Y_train)
dt_pred = clf_dt.predict(X_test)
dt_pred
accuracy_dt=round(accuracy_score(Y_test, dt_pred)*100,2)
print("Accuracy with Pruned Decision Tree Classifier :"+ str(accuracy_dt)+" %")
kappa_stats=round(metrics.cohen_kappa_score(dt_pred,Y_test),2)
print(kappa_stats)

Accuracy with Pruned Decision Tree Classifier :76.3 %
0.73


In [362]:
import pickle

pickle_out = open("dt11.pickle","wb")
pickle.dump(clf_dt, pickle_out)
pickle_out.close()

In [363]:
with open('dt-classifier.csv','a+') as f1:
    f1.write('dt11.pickle' +',')
    f1.write(str(accuracy_dt)+',')
    f1.write(str(kappa_stats))
    f1.write('\n')
f1.close

<function TextIOWrapper.close()>

In [65]:
results=confusion_matrix(Y_test, dt_pred)
print('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(Y_test, dt_pred))
print('Report : ')
print(classification_report(Y_test, dt_pred))


Confusion Matrix :
[[15  0  3  0  1  0  0  0  1]
 [ 1 16  2  0  0  1  0  0  0]
 [ 0  0 19  0  0  0  0  0  0]
 [ 0  0  4 10  0  2  2  2  0]
 [ 0  0  8  0 12  0  0  0  0]
 [ 0  1  0  1  0 11  0  0  5]
 [ 0  0  3  0  0  2 13  0  0]
 [ 0  1  0  0  0  0  0 17  0]
 [ 0  0  0  0  0  1  0  0 19]]
Accuracy Score : 0.7630057803468208
Report : 
              precision    recall  f1-score   support

           0       0.94      0.75      0.83        20
           1       0.89      0.80      0.84        20
           2       0.49      1.00      0.66        19
           3       0.91      0.50      0.65        20
           4       0.92      0.60      0.73        20
           5       0.65      0.61      0.63        18
           6       0.87      0.72      0.79        18
           7       0.89      0.94      0.92        18
           8       0.76      0.95      0.84        20

    accuracy                           0.76       173
   macro avg       0.81      0.76      0.76       173
weighted avg  

# Playing the competition between the two  players in two different classifiers  and comparing all the previous games  of that classifier with  the recent ones ...... classifier 11 with classifier 1 , 11 with 2, 11 with  3....................... 9,10 and so on.

In [ ]:
def UCT1(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            #state.DoMove(random.choice(state.GetMoves()))            
            model_dt = pickle.load(open("dt11.pickle","rb"))
            np_array=np.asarray(state.board)
            move=model_dt.predict(np_array.reshape(1,-1))
            #print(move)
            if move[0] in state.GetMoves():
                state.DoMove(move[0])
            else:
                state.DoMove(random.choice(state.GetMoves()))
            

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    #if verbose: print(rootnode.TreeToString(0))
    #else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

#___________________-------__________________________________________________________________________________________

def UCT2(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            #state.DoMove(random.choice(state.GetMoves()))            
            model_dt = pickle.load(open("dt1.pickle","rb"))
            np_array=np.asarray(state.board)
            move=model_dt.predict(np_array.reshape(1,-1))
            #print(move)
            if move[0] in state.GetMoves():
                state.DoMove(move[0])
            else:
                state.DoMove(random.choice(state.GetMoves()))
            

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    #if verbose: print(rootnode.TreeToString(0))
    #else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
